* https://github.com/KB-satou/GotchaGotcha/blob/master/SO3%E3%82%AF%E3%82%AA%E3%83%BC%E3%82%BF%E3%83%8B%E3%82%AA%E3%83%B3.ipynb
* https://twitter.com/KB_satou/status/1440604608405983245

In [1]:
using SymPy, LinearAlgebra

In [2]:
struct Quaternion{T}
  w::T
  x::T
  y::T
  z::T
end

In [3]:
Base.:*(p::Quaternion, q::Quaternion) = (
  Quaternion(p.w*q.w -p.x*q.x -p.y*q.y -p.z*q.z
    ,p.w*q.x + p.x*q.w + p.y*q.z -p.z*q.y 
    ,p.w*q.y + p.y*q.w + p.z*q.x -p.x*q.z 
    ,p.w*q.z + p.z*q.w + p.x*q.y -p.y*q.x 
    )
)

Base.:*(α, q::Quaternion) =  Quaternion( α*q.w, α*q.x, α*q.y, α*q.z)
Base.:+(p::Quaternion, q::Quaternion) = Quaternion(p.w+q.w, p.x+q.x, p.y+q.y, p.z+q.z)
Base.:-(p::Quaternion, q::Quaternion) = Quaternion(p.w-q.w, p.x-q.x, p.y-q.y, p.z-q.z)
Base.:(==)(p::Quaternion, q::Quaternion) = (p.w, p.x, p.y, p.z) == (q.w, q.x, q.y, q.z)

Base.show(io::IO, q::Quaternion) = show(io::IO, [q.w, q.x, q.y, q.z])
Base.show(io::IO, ::MIME"text/plain", q::Quaternion) = show(io::IO, MIME("text/plain"), [q.w, q.x, q.y, q.z])
Base.show(io::IO, ::MIME"text/latex", q::Quaternion{Sym}) = show(io, MIME("text/latex"), [q.w, q.x, q.y, q.z])

SymPy.expand(q::Quaternion) = Quaternion(expand(q.w), expand(q.x), expand(q.y), expand(q.z))

In [4]:
@syms w1::real w2::real x1::real x2::real y1::real y2::real z1::real z2::real 
@syms w3::real x3::real y3::real z3::real
@syms w::real x::real y::real z::real
@syms a::real b::real c::real
@syms a1::real b1::real c1::real
@syms a2::real b2::real c2::real
@syms α::real β::real
@syms θ::real

(θ,)

In [5]:
q = Quaternion(w,x,y,z)
q1 = Quaternion(w1,x1,y1,z1)
q2 = Quaternion(w2,x2,y2,z2)
q1 * q2

4-element Vector{Sym}:
 w1*w2 - x1*x2 - y1*y2 - z1*z2
 w1*x2 + w2*x1 + y1*z2 - y2*z1
 w1*y2 + w2*y1 - x1*z2 + x2*z1
 w1*z2 + w2*z1 + x1*y2 - x2*y1

## 線形性の確認

In [6]:
Q1 = expand(q*(α*q1 + β*q2))

4-element Vector{Sym}:
 w*w1*α + w*w2*β - x*x1*α - x*x2*β - y*y1*α - y*y2*β - z*z1*α - z*z2*β
 w*x1*α + w*x2*β + w1*x*α + w2*x*β + y*z1*α + y*z2*β - y1*z*α - y2*z*β
 w*y1*α + w*y2*β + w1*y*α + w2*y*β - x*z1*α - x*z2*β + x1*z*α + x2*z*β
 w*z1*α + w*z2*β + w1*z*α + w2*z*β + x*y1*α + x*y2*β - x1*y*α - x2*y*β

In [7]:
Q2 = expand(α*(q*q1) + β*(q*q2))

4-element Vector{Sym}:
 w*w1*α + w*w2*β - x*x1*α - x*x2*β - y*y1*α - y*y2*β - z*z1*α - z*z2*β
 w*x1*α + w*x2*β + w1*x*α + w2*x*β + y*z1*α + y*z2*β - y1*z*α - y2*z*β
 w*y1*α + w*y2*β + w1*y*α + w2*y*β - x*z1*α - x*z2*β + x1*z*α + x2*z*β
 w*z1*α + w*z2*β + w1*z*α + w2*z*β + x*y1*α + x*y2*β - x1*y*α - x2*y*β

In [8]:
Q1 == Q2

true

In [9]:
Base.adjoint(q::Quaternion) = Quaternion(q.w, -q.x, -q.y , -q.z)
Base.abs2(p::Quaternion) = p.w*p.w + p.x*p.x + p.y*p.y + p.z*p.z

In [10]:
q1*adjoint(q1)

4-element Vector{Sym}:
 w1^2 + x1^2 + y1^2 + z1^2
                         0
                         0
                         0

In [11]:
abs2(q1)

  2     2     2     2
w1  + x1  + y1  + z1 

In [12]:
abs2(q1*q2)

                               2                                  2           
(w1*w2 - x1*x2 - y1*y2 - z1*z2)  + (w1*x2 + w2*x1 + y1*z2 - y2*z1)  + (w1*y2 +

                       2                                  2
 w2*y1 - x1*z2 + x2*z1)  + (w1*z2 + w2*z1 + x1*y2 - x2*y1) 

In [13]:
expand(subs(expand(abs2(q1*q2)), w1^2 => 1-x1^2-y1^2-z1^2))

  2     2     2     2
w2  + x2  + y2  + z2 

In [14]:
subs(abs2(q1*q2), w1^2 => 1-x1^2-y1^2-z1^2)

                               2                                  2           
(w1*w2 - x1*x2 - y1*y2 - z1*z2)  + (w1*x2 + w2*x1 + y1*z2 - y2*z1)  + (w1*y2 +

                       2                                  2
 w2*y1 - x1*z2 + x2*z1)  + (w1*z2 + w2*z1 + x1*y2 - x2*y1) 

In [15]:
expand(subs(expand(abs2(q1*q2)), w1^2 +x1^2+y1^2+z1^2=> 1))

  2   2     2   2     2   2     2   2     2   2     2   2     2   2     2   2 
w1 *w2  + w1 *x2  + w1 *y2  + w1 *z2  + w2 *x1  + w2 *y1  + w2 *z1  + x1 *x2  

    2   2     2   2     2   2     2   2     2   2     2   2     2   2     2   
+ x1 *y2  + x1 *z2  + x2 *y1  + x2 *z1  + y1 *y2  + y1 *z2  + y2 *z1  + z1 *z2

2
 

In [16]:
subs(expand(abs2(q1*q2)), w1^2 => 1-x1^2-y1^2-z1^2)

  2   2     2   2     2   2     2 /    2     2     2    \     2   2     2   2 
w2 *x1  + w2 *y1  + w2 *z1  + w2 *\- x1  - y1  - z1  + 1/ + x1 *x2  + x1 *y2  

    2   2     2   2     2   2     2 /    2     2     2    \     2   2     2   
+ x1 *z2  + x2 *y1  + x2 *z1  + x2 *\- x1  - y1  - z1  + 1/ + y1 *y2  + y1 *z2

2     2   2     2 /    2     2     2    \     2   2     2 /    2     2     2  
  + y2 *z1  + y2 *\- x1  - y1  - z1  + 1/ + z1 *z2  + z2 *\- x1  - y1  - z1  +

  \
 1/

In [17]:
v = Quaternion(zero(Sym),a,b,c)

4-element Vector{Sym}:
 0
 a
 b
 c

In [18]:
q = Quaternion(w,x,y,z)

4-element Vector{Sym}:
 w
 x
 y
 z

In [19]:
expand(q*(v)*adjoint(q))

4-element Vector{Sym}:
                                                                      0
  a*w^2 + a*x^2 - a*y^2 - a*z^2 - 2*b*w*z + 2*b*x*y + 2*c*w*y + 2*c*x*z
  2*a*w*z + 2*a*x*y + b*w^2 - b*x^2 + b*y^2 - b*z^2 - 2*c*w*x + 2*c*y*z
 -2*a*w*y + 2*a*x*z + 2*b*w*x + 2*b*y*z + c*w^2 - c*x^2 - c*y^2 + c*z^2

In [20]:
(q*(v)*adjoint(q))

4-element Vector{Sym}:
 w*(-a*x - b*y - c*z) + x*(a*w - b*z + c*y) + y*(a*z + b*w - c*x) + z*(-a*y + b*x + c*w)
 w*(a*w - b*z + c*y) - x*(-a*x - b*y - c*z) + y*(-a*y + b*x + c*w) - z*(a*z + b*w - c*x)
 w*(a*z + b*w - c*x) - x*(-a*y + b*x + c*w) - y*(-a*x - b*y - c*z) + z*(a*w - b*z + c*y)
 w*(-a*y + b*x + c*w) + x*(a*z + b*w - c*x) - y*(a*w - b*z + c*y) - z*(-a*x - b*y - c*z)

In [21]:
expand(q*Quaternion(0,1,0,0)*adjoint(q))
expand(q*Quaternion(0,0,1,0)*adjoint(q))
expand(q*Quaternion(0,0,0,1)*adjoint(q))

4-element Vector{Sym}:
                     0
         2*w*y + 2*x*z
        -2*w*x + 2*y*z
 w^2 - x^2 - y^2 + z^2

In [22]:
Base.collect(q::Quaternion) = [q.w, q.x, q.y, q.z]

In [23]:
M = [collect(expand(q*Quaternion(0,1,0,0)*adjoint(q))) collect(expand(q*Quaternion(0,0,1,0)*adjoint(q))) collect(expand(q*Quaternion(0,0,0,1)*adjoint(q)))]

4×3 Matrix{Sym}:
                     0                      0                      0
 w^2 + x^2 - y^2 - z^2         -2*w*z + 2*x*y          2*w*y + 2*x*z
         2*w*z + 2*x*y  w^2 - x^2 + y^2 - z^2         -2*w*x + 2*y*z
        -2*w*y + 2*x*z          2*w*x + 2*y*z  w^2 - x^2 - y^2 + z^2

In [24]:
MtM = expand.(M' * M)

3×3 Matrix{Sym}:
 w^4 + 2*w^2*x^2 + 2*w^2*y^2 + 2*w^2*z^2 + x^4 + 2*x^2*y^2 + 2*x^2*z^2 + y^4 + 2*y^2*z^2 + z^4  …                                                                                              0
                                                                                             0                                                                                                 0
                                                                                             0     w^4 + 2*w^2*x^2 + 2*w^2*y^2 + 2*w^2*z^2 + x^4 + 2*x^2*y^2 + 2*x^2*z^2 + y^4 + 2*y^2*z^2 + z^4

In [25]:
expand.(subs.(MtM, w^2 => 1-x^2-y^2-z^2))

3×3 Matrix{Sym}:
 1  0  0
 0  1  0
 0  0  1

In [26]:
detM = det(M[2:end,:])

 6      4  2      4  2      4  2      2  4      2  2  2      2  2  2      2  4
w  + 3*w *x  + 3*w *y  + 3*w *z  + 3*w *x  + 6*w *x *y  + 6*w *x *z  + 3*w *y 

      2  2  2      2  4    6      4  2      4  2      2  4      2  2  2      2
 + 6*w *y *z  + 3*w *z  + x  + 3*x *y  + 3*x *z  + 3*x *y  + 6*x *y *z  + 3*x 

  4    6      4  2      2  4    6
*z  + y  + 3*y *z  + 3*y *z  + z 

In [27]:
expand.(subs(detM, w^2 => 1-x^2-y^2-z^2))

1

In [28]:
M2 = subs.(subs.(M,x,0),y,0)

4×3 Matrix{Sym}:
         0          0          0
 w^2 - z^2     -2*w*z          0
     2*w*z  w^2 - z^2          0
         0          0  w^2 + z^2

In [29]:
M3 = subs.(M2, w, cos(θ))

4×3 Matrix{Sym}:
               0                0               0
 -z^2 + cos(θ)^2      -2*z*cos(θ)               0
      2*z*cos(θ)  -z^2 + cos(θ)^2               0
               0                0  z^2 + cos(θ)^2

In [30]:
M4 = subs.(M3, z, sin(θ))

4×3 Matrix{Sym}:
                    0                     0                    0
 -sin(θ)^2 + cos(θ)^2      -2*sin(θ)*cos(θ)                    0
      2*sin(θ)*cos(θ)  -sin(θ)^2 + cos(θ)^2                    0
                    0                     0  sin(θ)^2 + cos(θ)^2

In [31]:
simplify.(M4)

4×3 Matrix{Sym}:
        0          0  0
 cos(2*θ)  -sin(2*θ)  0
 sin(2*θ)   cos(2*θ)  0
        0          0  1

In [32]:
M3 = subs.(M2, w, cos(θ/2))

4×3 Matrix{Sym}:
                 0                  0                 0
 -z^2 + cos(θ/2)^2      -2*z*cos(θ/2)                 0
      2*z*cos(θ/2)  -z^2 + cos(θ/2)^2                 0
                 0                  0  z^2 + cos(θ/2)^2

In [33]:
M4 = subs.(M3, z, sin(θ/2))

4×3 Matrix{Sym}:
                        0                         0                        0
 -sin(θ/2)^2 + cos(θ/2)^2      -2*sin(θ/2)*cos(θ/2)                        0
      2*sin(θ/2)*cos(θ/2)  -sin(θ/2)^2 + cos(θ/2)^2                        0
                        0                         0  sin(θ/2)^2 + cos(θ/2)^2

In [34]:
simplify.(M4)

4×3 Matrix{Sym}:
      0        0  0
 cos(θ)  -sin(θ)  0
 sin(θ)   cos(θ)  0
      0        0  1